In [11]:
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder \
    .appName("HELK Reader") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [13]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [14]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-*/")

In [15]:
sysmon_df.createOrReplaceTempView("sysmon_events")

In [20]:
Hook_Injection = spark.sql(
'''
SELECT 
    b.host_name,
    b.process_parent_name,
    a.process_target_name,
    a.process_granted_access,
    a.user_reporter_name
FROM sysmon_events b
JOIN sysmon_events a
    ON a.process_target_guid = b.process_guid
    AND a.event_id = 10
    AND a.process_granted_access = "2097151" -- Assuming they are asking for full privileges. Was hard to narrow down the minimum rights needed to perform. API's were not specefic. 
    AND b.process_parent_name = a.process_name
    AND a.process_name != b.process_name
JOIN sysmon_events c
    ON b.process_guid = c.process_guid
    AND c.event_id = 13
    AND c.process_guid = a.process_target_guid
    AND c.process_name = a.process_target_name 
    AND (substring_index(c.registry_key_path, '\\\\', -1)) = b.process_name
JOIN sysmon_events d
    ON d.process_guid = a.process_target_guid
    AND d.event_id = 7
    AND d.module_loaded = a.process_target_path
    AND LOWER(d.OriginalFileName) = (substring_index(c.registry_key_path, '\\\\', -1))
    AND d.OriginalFileName = b.OriginalFileName
WHERE
     b.event_id = 1
    AND b.process_name = LOWER(b.OriginalFileName)
    AND (NOT (b.process_name = "rundll32.exe" AND b.process_parent_name = "svchost.exe")) -- Blind spot + Assumption
    AND (NOT (b.process_name = "notepad.exe" AND b.process_parent_name = "explorer.exe")) -- Blind spot + Assumption
    AND (NOT (b.process_name = "devenv.exe" AND b.process_parent_name = "explorer.exe")) -- Blind spot + Assumption
    AND (NOT (b.process_name = "cmd.exe" AND b.process_parent_name = "explorer.exe")) -- Blind spot + Assumption
'''
).show()

+------------------+-------------------+-------------------+----------------------+------------------+
|         host_name|process_parent_name|process_target_name|process_granted_access|user_reporter_name|
+------------------+-------------------+-------------------+----------------------+------------------+
|win10.marvel.local|       explorer.exe|     powershell.exe|                  4096|            SYSTEM|
|win10.marvel.local|       explorer.exe|     powershell.exe|                  4096|            SYSTEM|
|win10.marvel.local|       explorer.exe|     powershell.exe|                  4096|            SYSTEM|
|win10.marvel.local|       explorer.exe|     powershell.exe|                  4096|            SYSTEM|
|win10.marvel.local|       explorer.exe|     powershell.exe|                  4096|            SYSTEM|
|win10.marvel.local|       explorer.exe|     powershell.exe|                  4096|            SYSTEM|
|win10.marvel.local|       explorer.exe|     powershell.exe|             

In [ ]:
# https://github.com/secrary/InjectProc